In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [2]:
dataset = spark.table("bank_full_bd3df_csv")
cols = dataset.columns

In [3]:
categoricalColumns = ["job", "marital", "education", "default", "housing", "loan", "contact", "month","poutcome"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [4]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="y", outputCol="label")
stages += [label_stringIdx]

In [5]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [6]:
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [7]:
# Fit model to prepped data
lrModel = LogisticRegression().fit(preppedDataDF)

# ROC for training data
display(lrModel, preppedDataDF, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9102080989617555
0.0,0.1111111111111111,0.9102080989617555
0.011904761904761904,0.1111111111111111,0.8727759997784728
0.011904761904761904,0.2222222222222222,0.7336536465107624
0.023809523809523808,0.2222222222222222,0.7259079653018172
0.03571428571428571,0.2222222222222222,0.6134736427382863
0.03571428571428571,0.3333333333333333,0.5949502237042397
0.047619047619047616,0.3333333333333333,0.5214377378376418
0.05952380952380952,0.3333333333333333,0.4940334048594755
0.07142857142857142,0.3333333333333333,0.3992205115280179


In [8]:
display(lrModel, preppedDataDF)

fitted values,residuals
-4.463155554981384,-0.011394601680481042
-2.5914586517811085,-0.06969015408985259
-4.970051147367216,-0.006894922898203429
-4.894938299514752,-0.007428771318361844
-4.464246210287158,-0.011382322249983277
-5.007168775932872,-0.006645360644032065
-4.730144389014285,-0.008747993919328786
-2.2227135270524596,-0.0977292688833315
-4.38058025598015,-0.012363327533481577
-3.3834761128355844,-0.032815887085159756


In [9]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
display(dataset)

label,features,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0.0,"List(0, 42, List(1, 11, 14, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 58.0, 2143.0, 5.0, 261.0, 1.0, -1.0))",58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
0.0,"List(0, 42, List(2, 12, 13, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 44.0, 29.0, 5.0, 151.0, 1.0, -1.0))",44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
0.0,"List(0, 42, List(7, 11, 13, 16, 17, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 33.0, 2.0, 5.0, 76.0, 1.0, -1.0))",33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
0.0,"List(0, 42, List(0, 11, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 47.0, 1506.0, 5.0, 92.0, 1.0, -1.0))",47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
0.0,"List(0, 42, List(12, 16, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 33.0, 1.0, 5.0, 198.0, 1.0, -1.0))",33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
0.0,"List(0, 42, List(1, 11, 14, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 35.0, 231.0, 5.0, 139.0, 1.0, -1.0))",35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
0.0,"List(0, 42, List(1, 12, 14, 16, 17, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 28.0, 447.0, 5.0, 217.0, 1.0, -1.0))",28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
0.0,"List(0, 42, List(7, 14, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 42.0, 2.0, 5.0, 380.0, 1.0, -1.0))",42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
0.0,"List(0, 42, List(5, 11, 15, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 58.0, 121.0, 5.0, 50.0, 1.0, -1.0))",58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
0.0,"List(0, 42, List(2, 12, 13, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 43.0, 593.0, 5.0, 55.0, 1.0, -1.0))",43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [10]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

31645
13566

In [11]:
# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [12]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [13]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & occupation
selected = predictions.select("label", "job", "education", "age")
display(selected)

label,job,education,age
0.0,blue-collar,secondary,27
0.0,blue-collar,secondary,28
0.0,blue-collar,secondary,28
0.0,blue-collar,secondary,28
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,30
0.0,blue-collar,secondary,30


In [14]:
# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

Out[ 22 ]: 0.903498241869877

In [15]:
evaluator.getMetricName()

Out[ 23 ]: 'areaUnderROC'

In [16]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [17]:
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [18]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [19]:
# Use test set to measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [20]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

Out[ 28 ]: 0.9033687768969302

In [21]:
print('Model Intercept: ', cvModel.bestModel.intercept)

Model Intercept: -2.051544917694723

In [22]:
weights = cvModel.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)

Feature Weight
-0.22222472762057235
-0.04162489880504778
-0.0024997744071399487
0.20147442863588244
-0.09340987917053635
0.4235262987344917
-0.1815665968419371
-0.17803569470157413
0.04541524873504915
-0.36081228145405847


In [23]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "job", "education", "age")
display(selected)

label,job,education,age
0.0,blue-collar,secondary,27
0.0,blue-collar,secondary,28
0.0,blue-collar,secondary,28
0.0,blue-collar,secondary,28
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,29
0.0,blue-collar,secondary,30
0.0,blue-collar,secondary,30


In [24]:
trainingSummary = lrModel.summary
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.8989097803760467
FPR: 0.6007680525108988
TPR: 0.8989097803760467
F-measure: 0.8842206865207111
Precision: 0.8826602557154871
Recall: 0.8989097803760467

In [25]:
from pyspark.sql.types import IntegerType

# notice the variable name (more below)
mylist = [0,42,[2,11,13,16,18,20,21,32,35,36,37,38,39,40],[1,1,1,1,1,1,1,1,53,-3,5,1666,1,-1]]


cvModel.transform(mylist)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-694866429795890> in <module> () 
 5 
 6 
 ----> 7 cvModel . transform ( mylist ) 

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 171 return self . copy ( params ) . _transform ( dataset ) 
 172 else : 
 --> 173 return self . _transform ( dataset ) 
 174 else : 
 175 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/tuning.py in _transform (self, dataset) 
 420 
 421 def _transform ( self , dataset ) : 
 --> 422 return self . bestModel . transform ( dataset ) 
 423 
 424 @ since ( "1.4.0" ) 

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 171 return self . copy ( params ) . _transform ( dataset ) 
 172 else : 
 --> 173 return self . _transform ( dataset ) 
 174 else : 
 175 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _transform (self, dataset) 
 310 def _transform ( self , dataset ) : 
 311 self . _transfer_params_to_java ( ) 
 --> 312 return DataFrame ( self . _java_obj . transform ( dataset . _jdf ) , dataset . sql_ctx ) 
 313 
 314 

 AttributeError : 'list' object has no attribute '_jdf'